<a href="https://colab.research.google.com/github/liranbd1/FaceMaskDetectionLab/blob/main/Mask_Detection_Prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Missing API's

* facenet-pytroch library for MTCNN

In [ ]:
!pip install facenet-pytorch

#Connecting to GDrive
All of the dataset and saving of the models are stored in GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# Imports

In [ ]:
import pandas as pd
import os
from facenet_pytorch.models.mtcnn import MTCNN
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import torchvision
from torchvision import transforms
from IPython.core.display import Math
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import os.path
import torch.optim as optim

#Parameters

In [ ]:
device = 'cude' if torch.cuda.is_available() else 'cpu'
model_weights_path = ""

mtcnn_model = MTCNN(keep_all = True).to(device)
classifier = OurClassifier().to(device)

classifier.load_state_dict(torch.load(model_weights_path))
classifier.eval()

transformations = transforms.Compose(
    [
      transforms.Resize((32, 32)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# Production function

Given an image path from the user our program:
1. Use MTCNN to detect faces bounding boxes in the image.
2. For each bounding box we crop the image and pass the face detect to our classifier.
3. Saving the bbox and the output of the classifier.
4. Drawing on the original image boxes for the faces we found

***Green box*** -> Face with mask \
***Red box*** -> Face without a mask

In [ ]:
def classify_img(img_path):
  img = Image.open(img_path)
  bboxes, probs = mtcnn.detect(img)

  if bboxes is None:
    print("No face detected")
    return
  
  results = []
  for box,prob in zip(bboxes, probs):
    
    if prob < 0.85:
      print("Skip probabilty below 85%")
      continue

    tuple_box = (cor for cor in box)
    face_detected = img.crop(tuple_box)
    face_tensor = transformations(face_detected).to(device)
    outputs = calssifier(face_tensor)
    prediction = torch.argmax(outputs, dim=1)
    results.append([box, prediction])

  fig, ax = plt.subplot()
  ax.imshow(img)
    
  for result in results:
    box, label = result
    anchor_point = (box[0], box[1])
    w = abs(box[0] - box[2])
    h = abs(box[1] - box[3])
    edge_color = 'r' if label == 1 else 'g'
    rect = patches.Rectangle(anchor_point, w,h , linewidth = 1, edgecolor = edge_color, 
                             facecolor = 'none')
    ax.add_patch(rect)

  plt.show()